# Problem 1 - Extending the Lady Tasting Tea Experiment

The aim of this notebook, is to extend the classical *Lady Tasting Tea* experiment, as demonstrated in the module, to use a larger number of cups.

To achieve this, I will:

- Set up the original 8-cup experiment
- Set up extended 12-cup experiment,
- Use Numpy, to simulate both the the original 8 cup and extended 12-cup experiment
- Compare the simulated probabilities
- Interpret the results

# Importing Required Libraries

In [1]:
# Mathematical functions from the standard library.
# https://docs.python.org/3/library/math.html
import math

# Permutations and combinations.
# https://docs.python.org/3/library/itertools.html
import itertools

# Random selections.
# https://docs.python.org/3/library/random.html
import random

# Numerical structures and operations.
# https://numpy.org/doc/stable/reference/index.html#reference
import numpy as np

# Plotting.
# https://matplotlib.org/stable/contents.html
import matplotlib.pyplot as plt

# Experiment Set Up

## Original Experiment - Cups of Tea

PLACEHOLDER - Describe original experiment

In [2]:
# --- Original 8-cup setup ---

no_cups_original = 8
no_milk_original = 4

# Number of ways to choose 4 cups from 8.
ways_original = math.comb(no_cups_original, no_milk_original)

# Exact probability of getting all 4 correct by chance.
prob_all_correct_original = 1 / ways_original

ways_original, prob_all_correct_original

(70, 0.014285714285714285)

## Updated Experiment - Cups of Tea

In [3]:
# --- Extended 12-cup setup ---

no_cups_extended = 12
no_milk_extended = 4

# Number of ways to choose 4 cups from 12.
ways_extended = math.comb(no_cups_extended, no_milk_extended)

# Exact probability of getting all 4 correct by chance.
prob_all_correct_extended = 1 / ways_extended

ways_extended, prob_all_correct_extended


(495, 0.00202020202020202)

# Running the Extended Experiment

PLACEHOLDER - Explain the extended experiment

In [4]:
# True positions of the 4 milk-first cups in the extended experiment.
# For simplicity, assume cups 0, 1, 2, and 3 are the milk-first cups.
true_milk_labels = set(range(no_milk_extended))

# Number of simulated experiments.
n_sims = 100_000

# NumPy random number generator with a fixed seed for reproducibility.
rng = np.random.default_rng(seed=123)

successes = 0

for _ in range(n_sims):
    # Randomly choose 4 distinct cups as the participant's guess.
    guessed_labels = rng.choice(no_cups_extended,
                                size=no_milk_extended,
                                replace=False)
    
    # Check if the guess matches the true milk-first cups exactly.
    if set(guessed_labels) == true_milk_labels:
        successes += 1

# Estimated probability from the simulation.
estimated_prob_all_correct_extended = successes / n_sims

successes, n_sims, estimated_prob_all_correct_extended


(194, 100000, 0.00194)